# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686550


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:06<01:31,  3.28s/it]

Rendering models:  10%|█         | 3/30 [00:06<01:03,  2.36s/it]

Rendering models:  13%|█▎        | 4/30 [00:06<00:44,  1.70s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:31,  1.24s/it]

...

Rendering models:  90%|█████████ | 27/30 [00:10<00:00,  6.93it/s]

Rendering models:  93%|█████████▎| 28/30 [00:10<00:00,  6.93it/s]

Rendering models:  97%|█████████▋| 29/30 [00:10<00:00,  5.40it/s]

Rendering models: 100%|██████████| 30/30 [00:11<00:00,  4.86it/s]

not-logged-in-df8268e1657cfc2c5472    1.034404
zac8151                               0.000645
not-logged-in-c4869230b979ccc7c666    0.000458
SavinSpaceEngineer                    0.003559
djswanso                              0.000471
kjkavene                              0.000892
not-logged-in-3bc5cf64460c02716efb    0.000880
justsomenoodles                       0.000441
jmfranci                              0.001142
not-logged-in-9a813f8c400d5fedde5d    0.024050
not-logged-in-5f30b2bb9b7f9d49da41    0.059770
not-logged-in-5f30b2bb9b7f9d49da41    0.055137
iesfranciscoayala                     0.000918
ComeMyChild                           0.071059
peakscience7                          0.027645
rhondacard                            0.004069
Lavadude                              0.020050
craftmole                             0.000551
atayl233                              0.001509
Alissa18                              0.002688
swalla46                              0.000430
h4ranek      

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())